In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jobathon-nov-2022/sample_submission.csv
/kaggle/input/jobathon-nov-2022/train.csv
/kaggle/input/jobathon-nov-2022/test.csv


In [2]:
!pip install pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 3.6 MB/s eta 0:00:00


In [3]:
train=pd.read_csv('/kaggle/input/jobathon-nov-2022/train.csv')
print(train.shape)

(94992, 3)


In [4]:
test=pd.read_csv('/kaggle/input/jobathon-nov-2022/test.csv')
print(test.shape)

(26304, 2)


In [5]:
train.head()

,row_id,datetime,energy
0,1,2008-03-01 00:00:00,1259.985563
1,2,2008-03-01 01:00:00,1095.541500
2,3,2008-03-01 02:00:00,1056.247500
3,4,2008-03-01 03:00:00,1034.742000
4,5,2008-03-01 04:00:00,1026.334500


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94992 entries, 0 to 94991
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   row_id    94992 non-null  int64  
 1   datetime  94992 non-null  object 
 2   energy    93092 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.2+ MB


In [7]:
train['energy'].describe()

count    93092.000000
mean      1702.995014
std        369.322103
min        866.388600
25%       1431.541100
50%       1654.465800
75%       1935.993450
max       3374.399600
Name: energy, dtype: float64

In [8]:
train['datetime']=pd.to_datetime(train['datetime'],infer_datetime_format=True)

In [9]:
train['datetime'].min(),train['datetime'].max()

(Timestamp('2008-03-01 00:00:00'), Timestamp('2018-12-31 23:00:00'))

In [10]:
train['year']=train['datetime'].dt.year
train['month']=train['datetime'].dt.month
train['day']=train['datetime'].dt.day
train['hour']=train['datetime'].dt.hour

In [11]:
train['year'].unique()

array([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018])

In [12]:
test['datetime']=pd.to_datetime(test['datetime'],infer_datetime_format=True)

In [13]:
test['datetime'].min(),test['datetime'].max()

(Timestamp('2019-01-01 00:00:00'), Timestamp('2021-12-31 23:00:00'))

In [14]:
test['year']=test['datetime'].dt.year
test['month']=test['datetime'].dt.month
test['day']=test['datetime'].dt.day
test['hour']=test['datetime'].dt.hour

Split Train into train and validation sets

In [15]:
import datetime 
train_start = datetime.datetime(year=2017,month=11,day=1,hour=0)
val_start = datetime.datetime(year=2018,month=1,day=1,hour=0)
val_end = datetime.datetime(year=2018,month=12,day=31,hour=23)
# val_start = datetime.datetime(year=2016,month=1,day=1,hour=0)
# val_end = datetime.datetime(year=2018,month=12,day=31,hour=23)
X_val= train[(train['datetime']>=val_start) & (train['datetime']<=val_end)].copy()
X_train= train[(train['datetime']>=train_start) & (train['datetime']<val_start)].copy()
print(X_train.shape)
print(X_val.shape)
X_val.head()           

(1464, 7)
(8760, 7)


,row_id,datetime,energy,year,month,day,hour
86232,86233,2018-01-01 00:00:00,1642.7628,2018,1,1,0
86233,86234,2018-01-01 01:00:00,NaN,2018,1,1,1
86234,86235,2018-01-01 02:00:00,1463.6552,2018,1,1,2
86235,86236,2018-01-01 03:00:00,1413.1208,2018,1,1,3
86236,86237,2018-01-01 04:00:00,1385.8796,2018,1,1,4


In [16]:
targetcol = 'energy'
val_target = X_val[targetcol]
print(X_val[targetcol].isnull().sum())

174


In [17]:
val_target.fillna(X_train[targetcol].mean(),inplace=True)

In [18]:
from sklearn.metrics import mean_squared_error

In [19]:
#compute baseline error by predicting train energy mean as the energy for all time
val_preds_baseline = np.full(len(X_val),train[targetcol].mean())

In [20]:
print(X_train[targetcol].isnull().sum())
X_train.fillna(X_train[targetcol].mean(),inplace=True)
print(X_train[targetcol].isnull().sum())

33
0


In [21]:
print(val_target[0:5])

86232    1642.762800
86233    1973.609798
86234    1463.655200
86235    1413.120800
86236    1385.879600
Name: energy, dtype: float64


In [22]:
#compute error score on predictions
val_score = mean_squared_error(val_target,val_preds_baseline,squared=False)
print('valid score:',val_score)
#458.85 for median baseline
#427.85 for mean baseline
#857 for auto arima
#203 for auto arima seasonality

#381.12 for 14 days for mean baseline  197 auto arima
#340 for 90 days for mean baseline  289 auto arima
#482 for 365 days for mean baseline 349 auto arima

valid score: 482.5800266525437


In [23]:
# valid score: 418.17182076917203

Test Predictions Baseline

In [24]:
test_preds_baseline = np.full(len(test),train[targetcol].mean())

In [25]:
subm = pd.DataFrame()
subm['row_id'] = test['row_id']
subm[targetcol]=test_preds_baseline
subm.to_csv('submission.csv',index=False)

In [26]:
train['row_id'].describe()

count    94992.000000
mean     47496.500000
std      27421.972723
min          1.000000
25%      23748.750000
50%      47496.500000
75%      71244.250000
max      94992.000000
Name: row_id, dtype: float64

In [27]:
#check saved s.absbmission data
pd.read_csv('submission.csv').head()

,row_id,energy
0,94993,1702.995014
1,94994,1702.995014
2,94995,1702.995014
3,94996,1702.995014
4,94997,1702.995014


Auto ARIMA model

In [28]:
# import pmdarima as pm

In [29]:
# X_train.tail(25)

In [30]:
# %%time
# # auto_arima = pm.auto_arima(X_train[[targetcol]], n_jobs =-1,
# #                            stepwise=False, seasonal=False)
# auto_arima = pm.auto_arima(X_train[[targetcol]], n_jobs =-1,
#                            stepwise=False, seasonal=True,m=24)
# auto_arima

In [31]:
# auto_arima.summary()

In [32]:
# %%time
# forecast_test_auto = auto_arima.predict(n_periods=len(X_val))
# # val_forecast_auto = [None]*len(X_train) + list(forecast_test_auto)
# val_forecast_auto = np.array(list(forecast_test_auto))

In [33]:
# #compute error score on predictions
# val_score = mean_squared_error(val_target,val_forecast_auto,squared=False)
# print('valid score:',val_score)